In [36]:
%pip install -r requirements.txt

Processing c:\home\conda\feedstock_root\build_artifacts\asttokens_1698341106958\work (from -r requirements.txt (line 2))
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\home\\conda\\feedstock_root\\build_artifacts\\asttokens_1698341106958\\work'



## Путь, где хранится датасет

In [37]:
import pathlib

dataset_dir = pathlib.Path('./dataset')

## Параметры картинок в датасете

### Высота и ширина картинки (разрешение)

In [38]:
img_height = 32
img_width = 32

### Количество цветовых каналов

In [39]:
color_channels = 3

### Сколько проб использовать для обновления весов за раз

In [40]:
batch_size = 32

### Число элементов в первом скрытом слое
Количество элементов в слое имеет зависимость `Ns*2^(n-1)`, где n - номер скрытого слоя

In [41]:
Ns = 16

### Kernel size (на какие кусочки Conv2D разбивает входные данные)
`kernel_size = 3` означает разбиение на кусочки 3x3 px

In [42]:
kernel_size = 3

### Кол-во итераций обучения

In [43]:
epochs = 10

## Загрузка датасета в train dataset и validation dataset
Отношение `4:1`

In [44]:
import keras

image_count = len(list(dataset_dir.glob('*/*.png')))

train_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    color_mode="rgb",
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    color_mode="rgb",
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 57627 files belonging to 66 classes.
Using 46102 files for training.
Found 57627 files belonging to 66 classes.
Using 11525 files for validation.


## Замена названий классов (букв) на human readable (а, б, в, ...)

In [45]:
import numpy as np

char = ord('а')
char_upper = ord('А')
yo = False
Yo = False

print("До: " + str(train_ds.class_names))

for i, v in enumerate(train_ds.class_names):
    if train_ds.class_names[i].startswith('00'):
       if (char == ord("ж") and not yo):
           train_ds.class_names[i] = 'ё'
           yo = True
           continue
       train_ds.class_names[i] = chr(char)
       char += 1
    else:
       if (char_upper == ord("Ж") and not Yo):
           train_ds.class_names[i] = 'Ё'
           Yo = True
           continue
       train_ds.class_names[i] = chr(char_upper)
       char_upper += 1

class_names = train_ds.class_names

print("После: " + str(class_names))


До: ['00_00_00', '00_01_00', '00_02_00', '00_03_00', '00_04_00', '00_05_00', '00_06_00', '00_07_00', '00_08_00', '00_09_00', '00_10_00', '00_11_00', '00_12_00', '00_13_00', '00_14_00', '00_15_00', '00_16_00', '00_17_00', '00_18_00', '00_19_00', '00_20_00', '00_21_00', '00_22_00', '00_23_00', '00_24_00', '00_25_00', '00_26_00', '00_27_00', '00_28_00', '00_29_00', '00_30_00', '00_31_00', '00_32_00', '01_00_00', '01_01_00', '01_02_00', '01_03_00', '01_04_00', '01_05_00', '01_06_00', '01_07_00', '01_08_00', '01_09_00', '01_10_00', '01_11_00', '01_12_00', '01_13_00', '01_14_00', '01_15_00', '01_16_00', '01_17_00', '01_18_00', '01_19_00', '01_20_00', '01_21_00', '01_22_00', '01_23_00', '01_24_00', '01_25_00', '01_26_00', '01_27_00', '01_28_00', '01_29_00', '01_30_00', '01_31_00', '01_32_00']
После: ['а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 

Сохраним названия классов для pretty предикта

In [46]:
np.savetxt('class_names-dataset.txt', class_names, fmt='%s')

### Определение слоев нейронной сети и компиляция модели

In [47]:
import tensorflow as tf
import keras.layers as layers

num_classes = len(class_names)

model = keras.models.Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, colors)),
  layers.Conv2D(Ns, kernel_size, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(Ns*2, kernel_size, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(Ns*2**2, kernel_size, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(Ns*2**3, kernel_size, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  #layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(Ns*2**4, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Характеристики слоев модели

In [48]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_3 (Rescaling)         │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 32, 32, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 16, 16, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 8, 8, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 66)             │        16,962 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 245,730 (959.88 KB)

 Trainable params: 245,730 (959.88 KB)

 Non-trainable params: 0 (0.00 B)

### Обучение модели и сохранение

In [49]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

model.save('russian-cursive-32x32rgb.model.keras')

Epoch 1/10
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.1787 - loss: 3.3742 - val_accuracy: 0.7110 - val_loss: 0.9792
Epoch 2/10
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - accuracy: 0.7480 - loss: 0.8486 - val_accuracy: 0.8211 - val_loss: 0.5872
Epoch 3/10
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - accuracy: 0.8433 - loss: 0.5117 - val_accuracy: 0.8567 - val_loss: 0.4703
Epoch 4/10
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.8810 - loss: 0.3794 - val_accuracy: 0.8715 - val_loss: 0.4157
Epoch 5/10
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9061 - loss: 0.2943 - val_accuracy: 0.8994 - val_loss: 0.3365
Epoch 6/10
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9237 - loss: 0.2375 - val_accuracy: 0.8841 - val_loss: 0.3968
Epoch 7/10
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - accuracy: 0.9349 - loss: 0.2013 - val_accuracy: 0.9107 - val_loss: 0.3295
Epoch 8/10
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.9384 -